In [1]:
import pandas as pd
from sklearn.model_selection import ParameterGrid
from neuralprophet import NeuralProphet, set_log_level, set_random_seed
from features import add_stock_price_feature

# Disable logging messages unless there is an error
set_log_level("ERROR")

df = pd.read_csv('2330_stock_data.csv', parse_dates=['ds'])
df = add_stock_price_feature(df)
# plt = df.plot(x="ds", y="y", figsize=(15, 5))
df.info()

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Unnamed: 0                570 non-null    int64         
 1   ds                        570 non-null    datetime64[ns]
 2   open_price                570 non-null    float64       
 3   high_price                570 non-null    float64       
 4   low_price                 570 non-null    float64       
 5   y                         570 non-null    float64       
 6   volume                    570 non-null    int64         
 7   foreign                   570 non-null    int64         
 8   investment                570 non-null    int64         
 9   dealer                    570 non-null    int64         
 10  ratio_over_400_shares     570 non-null    float64       
 11  shareholders_400_to_600   570 non-null    int64         
 12  shareholders_600_to_80

In [2]:
from utils import val_mape

param_grid = {
    'foreign': [0, 5, 10],
    'investment': [0, 5, 10],
    'dealer': [0, 5, 10]
}

results = []
# Iterate over each combination of hyperparameters
for params in ParameterGrid(param_grid):
    # Initialize the NeuralProphet model with current hyperparameters
    print([params])
    m = NeuralProphet(
        yearly_seasonality=False,
        weekly_seasonality=True,
        n_lags=3,
    )
    m = m.add_country_holidays("TW")
    
    lag_regs = (
        # ('volume', 8),
        # ('high_low_diff', 10),
        ('foreign', params['foreign']),
        ('investment', params['investment']),
        ('dealer', params['dealer'])
    )
    columes = ['ds', 'y']
    for col, lag in lag_regs:
        if lag > 0:
            m.add_lagged_regressor(col, n_lags=lag)
            columes.append(col)
    
    df_train, df_val = m.split_df(df[columes], valid_p=0.2)
    set_random_seed(0)
    metrics = m.fit(df_train, validation_df=df_val)
    
    # Create a new dataframe reaching 365 into the future for our forecast, n_historic_predictions also shows historic data
    df_future = m.make_future_dataframe(df[columes], n_historic_predictions=True, periods=7)

    # Predict the future
    forecast = m.predict(df_future)
    
    rmse = metrics.iloc[-1]['RMSE_val']
    mape = val_mape(df_val, forecast)
    results.append({**params, 'RMSE': rmse, 'MAPE': mape})

[{'dealer': 0, 'foreign': 0, 'investment': 0}]


WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)



Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr:   0%|          | 0/219 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 14. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.



...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd

Predicting: |          | 0/? [00:00<?, ?it/s]

In [6]:
results_df = pd.DataFrame(results)
results_df.to_csv('params.csv')
results_df

,dealer,foreign,investment,RMSE,MAPE
0,0,0,0,18.046957,0.013945
1,0,0,5,18.059231,0.013823
2,0,0,10,18.391947,0.014329
3,0,5,0,18.786808,0.014303
4,0,5,5,18.701750,0.014242
5,0,5,10,18.757025,0.014435
6,0,10,0,19.065233,0.014605
7,0,10,5,18.756863,0.014392
8,0,10,10,18.573977,0.014358
9,5,0,0,19.869648,0.015186


In [7]:
results_df.sort_values(by="RMSE").head(10)

,dealer,foreign,investment,RMSE,MAPE
0,0,0,0,18.046957,0.013945
1,0,0,5,18.059231,0.013823
2,0,0,10,18.391947,0.014329
8,0,10,10,18.573977,0.014358
4,0,5,5,18.701750,0.014242
7,0,10,5,18.756863,0.014392
5,0,5,10,18.757025,0.014435
3,0,5,0,18.786808,0.014303
23,10,5,10,18.957312,0.014491
6,0,10,0,19.065233,0.014605


In [8]:
results_df.sort_values(by="MAPE").head(10)

,dealer,foreign,investment,RMSE,MAPE
1,0,0,5,18.059231,0.013823
0,0,0,0,18.046957,0.013945
4,0,5,5,18.701750,0.014242
3,0,5,0,18.786808,0.014303
2,0,0,10,18.391947,0.014329
8,0,10,10,18.573977,0.014358
7,0,10,5,18.756863,0.014392
5,0,5,10,18.757025,0.014435
23,10,5,10,18.957312,0.014491
6,0,10,0,19.065233,0.014605
